In [36]:
import tkinter as tk
from tkinter import Message ,Text 
import cv2,os
import shutil
import csv
import numpy as np
from PIL import Image, ImageTk
import pandas as pd
import datetime
import time
import tkinter.ttk as ttk
import tkinter.font as font

In [37]:
window = tk.Tk()
window.title("Face_Recogniser")
dialog_title = 'QUIT'
dialog_text = 'Are you sure?'
window.configure(background='#808080')


window.grid_rowconfigure(0, weight=1)
window.grid_columnconfigure(0, weight=1)


message = tk.Label(window, text="ATTENDANCE   USING   FACE   RECOGNITION " ,bg="#808080"  ,fg="white"  ,width=50  ,height=2,font=('rockwell', 30, ' bold')) 

message.place(x=200, y=0)
messagel = tk.Label(window, text="New Student / Not Enrolled                                                         Mark Attendance here" ,bg="#808080"  ,fg="white"  ,width=60+5  ,height=2,font=('times', 30, 'italic bold ')) 

messagel.place(x=0, y=150)
message2 = tk.Label(window, text="------------------------------------------------------------------------------------------------------------------------------" ,bg="#808080"  ,fg="white"  ,width=70  ,height=1,font=('times', 30, 'italic ')) 

message2.place(x=0, y=220)
message3 = tk.Label(window, text="------------------------------------------------------------------------------------------------------------------------------" ,bg="#808080"  ,fg="white"  ,width=70  ,height=1,font=('times', 30, 'italic ')) 

message3.place(x=0, y=100)



lbl = tk.Label(window, text="Enter Roll No.",width=20  ,height=1  ,fg="BLACK"  ,bg="#D3D3D3" ,font=('times', 15, ' bold ') ) 
lbl.place(x=0, y=200+100)

txt = tk.Entry(window,width=20  ,bg="#D3D3D3" ,fg="black",font=('times', 18, ' bold '))
txt.place(x=250, y=200+100)

lbl2 = tk.Label(window, text="Enter Name",width=20  ,fg="BLACK"  ,bg="#D3D3D3"    ,height=1 ,font=('times', 15, ' bold ')) 
lbl2.place(x=0, y=300+100)

txt2 = tk.Entry(window,width=20  ,bg="#D3D3D3"  ,fg="black",font=('times', 18, ' bold ')  )
txt2.place(x=250, y=300+100)

lbl3 = tk.Label(window, text="Notification : ",width=18-4 ,fg="BLACK"  ,bg="#D3D3D3"  ,height=2 ,font=('times', 15, ' bold')) 
lbl3.place(x=0, y=400+100)

message = tk.Label(window, text="" ,bg="#D3D3D3"  ,fg="red"  ,width=30  ,height=2, activebackground = "#D3D3D3" ,font=('times', 15, ' bold ')) 
message.place(x=200-18, y=400+100)

lbl3 = tk.Label(window, text="Attendance : ",width=20  ,fg="BLACK"  ,bg="#D3D3D3"  ,height=2 ,font=('times', 15, 'bold')) 
lbl3.place(x=1100, y=200+100)


message2 = tk.Label(window, text="" ,fg="black"   ,bg="#D3D3D3",activeforeground = "green",width=35  ,height=8  ,font=('times', 15, ' bold ')) 
message2.place(x=950+60, y=300-30+100)
 


In [38]:
def clear():
    txt.delete(0, 'end')    
    res = ""
    message.configure(text= res)
    

def clear2():
    txt2.delete(0, 'end')    
    res = ""
    message.configure(text= res)    
    
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False
 


In [39]:
def TakeImages():        
    Id=(txt.get())
    name=(txt2.get())
    if(is_number(Id) and name.isalpha()):
        cam = cv2.VideoCapture(0)
        harcascadePath = "haarcascade_frontalface_default.xml"
        detector=cv2.CascadeClassifier(harcascadePath)
        sampleNum=0
        while(True):
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)        
                #incrementing sample number 
                sampleNum=sampleNum+1
                #saving the captured face in the dataset folder TrainingImage
                cv2.imwrite("TrainingImage\ "+name +"."+Id +'.'+ str(sampleNum) + ".jpg", gray[y:y+h,x:x+w])
                #display the frame
                cv2.imshow('frame',img)
            #wait for 100 miliseconds 
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
            # break if the sample number is morethan 100
            elif sampleNum>60:
                break
        cam.release()
        cv2.destroyAllWindows() 
        res = "Images Saved for ID : " + Id +" Name : "+ name
        row = [Id , name]
        with open('StudentDetails\StudentDetails.csv','a+') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        csvFile.close()
        message.configure(text= res)
    else:
        if(is_number(Id)):
            res = "Enter Alphabetical Name"
            message.configure(text= res)
        if(name.isalpha()):
            res = "Enter Numeric Id"
            message.configure(text= res)
    


In [40]:
def TrainImages():
    recognizer = cv2.face_LBPHFaceRecognizer.create()#recognizer = cv2.face.LBPHFaceRecognizer_create()#$cv2.createLBPHFaceRecognizer()
    harcascadePath = "haarcascade_frontalface_default.xml"
    detector =cv2.CascadeClassifier(harcascadePath)
    faces,Id = getImagesAndLabels("TrainingImage")
    recognizer.train(faces, np.array(Id))
    recognizer.save("TrainingImageLabel\Trainner.yml")
    res = "Image Trained"#+",".join(str(f) for f in Id)
    message.configure(text= res)



In [41]:
def getImagesAndLabels(path):
    #get the path of all the files in the folder
    imagePaths=[os.path.join(path,f) for f in os.listdir(path)] 
    #print(imagePaths)
    
    #create empth face list
    faces=[]
    #create empty ID list
    Ids=[]
    #now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        #loading the image and converting it to gray scale
        pilImage=Image.open(imagePath).convert('L')
        #Now we are converting the PIL image into numpy array
        imageNp=np.array(pilImage,'uint8')
        #getting the Id from the image
        Id=int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces.append(imageNp)
        Ids.append(Id)        
    return faces,Ids



In [42]:
def TrackImages():
    recognizer = cv2.face.LBPHFaceRecognizer_create()#cv2.createLBPHFaceRecognizer()
    recognizer.read("TrainingImageLabel\Trainner.yml")
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath);    
    df=pd.read_csv("StudentDetails\StudentDetails.csv")
    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX        
    col_names =  ['Id','Name','Date','Time']
    attendance = pd.DataFrame(columns = col_names)    
    while True:
        ret, im =cam.read()
        gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        faces=faceCascade.detectMultiScale(gray, 1.2,5)    
        for(x,y,w,h) in faces:
            cv2.rectangle(im,(x,y),(x+w,y+h),(225,0,0),2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])                                   
            if(conf < 50):
                ts = time.time()      
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa=df.loc[df['Id'] == Id]['Name'].values
                tt=str(Id)+"-"+aa
                attendance.loc[len(attendance)] = [Id,aa,date,timeStamp]
                
            else:
                Id='Unknown'                
                tt=str(Id)  
            if(conf > 75):
                noOfFile=len(os.listdir("ImagesUnknown"))+1
                cv2.imwrite("ImagesUnknown\Image"+str(noOfFile) + ".jpg", im[y:y+h,x:x+w])            
            cv2.putText(im,str(tt),(x,y+h), font, 1,(255,255,255),2)        
        attendance=attendance.drop_duplicates(subset=['Id'],keep='first')    
        cv2.imshow('im',im) 
        if (cv2.waitKey(1)==ord('q')):
            break
    ts = time.time()      
    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
    Hour,Minute,Second=timeStamp.split(":")
    fileName="Attendance\Attendance_"+date+"_"+Hour+"-"+Minute+"-"+Second+".csv"
    attendance.to_csv(fileName,index=False)
    cam.release()
    cv2.destroyAllWindows()
    print(attendance)
    res=attendance
    message2.configure(text= res)




In [43]:
clearButton = tk.Button(window, text="X", command=clear  ,fg="black"  ,bg="#D3D3D3"  ,width=3  ,height=1 ,activebackground = "green" ,font=('times', 15, ' bold '))
clearButton.place(x=500, y=198+100)
clearButton2 = tk.Button(window, text="X", command=clear2  ,fg="black"  ,bg="#D3D3D3"  ,width=3  ,height=1, activebackground = "green" ,font=('times', 15, ' bold '))
clearButton2.place(x=500, y=298+100)    
takeImg = tk.Button(window, text="Take Images", command=TakeImages  ,fg="Red"  ,bg="#D3D3D3"  ,width=20  ,height=3, activebackground = "Green" ,font=('times', 15, ' bold '))
takeImg.place(x=0, y=500+100)
trainImg = tk.Button(window, text="Train Images", command=TrainImages  ,fg="red"  ,bg="#D3D3D3"  ,width=20  ,height=3, activebackground = "Green" ,font=('times', 15, ' bold '))
trainImg.place(x=300, y=500+100)
trackImg = tk.Button(window, text="Mark Attendance", command=TrackImages  ,fg="red"  ,bg="#D3D3D3"  ,width=20  ,height=3, activebackground = "Green" ,font=('times', 15, ' bold '))
trackImg.place(x=900, y=500+100)
quitWindow = tk.Button(window, text="Quit", command=window.destroy  ,fg="red"  ,bg="#D3D3D3"  ,width=20  ,height=3, activebackground = "Red" ,font=('times', 15, ' bold '))
quitWindow.place(x=1200, y=500+100)
copyWrite = tk.Text(window, background=window.cget("background"), borderwidth=0,font=('times', 30, 'italic bold underline'))
copyWrite.tag_configure("superscript", offset=10)
copyWrite.insert("insert", "Group no. 16 ")
copyWrite.configure(state="disabled",fg="blue"  )
copyWrite.pack(side="right")
copyWrite.place(x=800, y=750)


from time import strftime 
def times(): 
    string = "\n" + strftime('%H:%M:%S %p')
    string2=  strftime("%b-%d-%Y ")
    lbl.config(text = string2+string) 
    #lbl2.config(text= )
    lbl.after(1000, times) 

# Styling the label widget so that clock 
# will look more attractive 
lbl = tk.Label(window, font = ('calibri', 25, 'bold'),background = '#b19cd9', foreground = 'white',borderwidth=2, relief="raised") 

# Placing clock at the centre 
# of the tkinter window 
lbl.pack(side="right")
lbl.place(x=670,y=150)
times()


window.mainloop()

   Id      Name        Date      Time
0  75  [Mahesh]  2020-10-22  22:41:52
   Id        Name        Date      Time
0  72  [akshansh]  2020-10-22  22:47:55
1  60   [utkarsh]  2020-10-22  22:47:58
2  75    [Mahesh]  2020-10-22  22:48:04
